In [1]:
import numpy as np
import pandas as pd

In [2]:
csv_base = './csv/'

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

### Parámetros de entrada
Información que obtiene el modelo del sistema de información

In [3]:
# Demanda
demanda = 100
# Costo por unidad de tiempo
ctiempo = 1000

# Estos datos pueden ser obtenidos al consultar una vista en la base de datos
params_df = pd.read_csv(csv_base + 'entry_params.csv')
params_df.head()

,id_acopio,stock,precio,ppotencial,tiempo_alistam,ctransp,tiempo_transp
0,Santa Marta,30,100,20,2,3000,1
1,La Guajira,90,120,20,2,12000,4
2,Cordoba,10,105,40,2,6000,3


### Variables
Definición de variables para el modelo de optimización

In [4]:
N = params_df.shape[0]
# bounds = tuple((0, x_i) for x_i in params_df['stock'])

# Función objetivo
$$
\begin{align*}
    \sum_{i=0 \quad i\neq p}^{N} &\big[ \, kCA_i \times Precio(CA_i) + kCA_i \times cTransp(CA_i) + Tiempo(CA_i) \times cTiempo \, \big] \, + \\
                                 &\big[ \, kCA_p \times Precio(CA_p) + Demanda \times cTransp(CA_p) + Tiempo(CA_p) \times cTiempo \, \big]
\end{align*}
$$
Sujeto a las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{N} kCA_i &= Demanda \\
    kCA_i &\leq Stock(CA_i) + Ppotencial(CA_i) &\therefore \, i=0,\cdots ,N \\
    TiempoAlistam(CA_i) &\leq TiempoMaxDefinido &\therefore \, i=0,\cdots ,N \\
    Tiempo(CA_i) &= TiempoAlistam(CA_i) + TiempoTransp(CA_i) \\
\end{align*}
$$

In [5]:
import random

def f(x):
    delta = 0
    # p = random.randint(0, N-1) 
    p = 0
    
    for i in range(N):
        if i == p or x[i] == 0:
            continue
        
        delta += get_delta(x, i)
    
    delta += get_delta(x, p, True)
    
    return delta

def get_delta(x, idx, p=False):
    kca = x[idx]
    precio = params_df['precio'].iloc[idx]
    ctransp = params_df['ctransp'].iloc[idx]
    tiempo = params_df['tiempo_transp'].iloc[idx] + params_df['tiempo_alistam'].iloc[idx]
    
    if not p:
        return (kca * precio) + (kca * ctransp) + (tiempo * ctiempo) 
    else:
        return (kca * precio) + (demanda * ctransp) + (tiempo * ctiempo)
    
        
f((50, 0, 10))

374050

## scipy

**LinearConstraint**: $ lb \leq A \cdot v[x] \leq ub $

**NonLinearConstraint**: $ lb \leq g(x) \leq ub $

In [6]:
# c = tuple(idx for (_, idx) in bounds)
# Aeq = np.ones((1, len(c)))
# beq = 1

In [7]:
# from scipy.optimize import linprog
# # from scipy.optimize import differential_evolution
# 
# result = linprog(
#     c,
#     A_ub=Aeq,
#     b_ub=beq,
#     bounds=bounds,
#     method='highs',
#     integrality=1
# )
# print(result)

## pymoo

In [60]:
from pymoo.core.problem import ElementwiseProblem

xl = np.zeros(N)
xu = np.asarray([x_i for x_i in params_df['stock']])

class Queso(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=len(xl),
            n_obj=1,
            n_eq_constr=1,
            # n_ieq_constr=1,
            xl=0,
            xu=xu,
            vtype=int
        )
        
    def _evaluate(self, x, out, *args, **kwargs):
        print(f'{x}\n')
        out['F'] = f(x)
        out['H'] = demanda - np.sum(x)

In [61]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.termination import get_termination

problem = Queso()

algorithm = GA(
    # pop_size=9,
    sampling=IntegerRandomSampling(),
    eliminate_duplicates=True
)

# termination = get_termination('time', '00:00:05')

res = minimize(problem,
               algorithm,
               # termination,
               seed=1,
               verbose=False)

print(f'F: {res.F}, \nX: {res.X}')

[ 5 83  5]

[11 38  5]

[12  8  4]

[ 8 32  5]

[ 9 34  8]

[11 10  5]

[ 5 23  8]

[15 15  1]

[ 0 87  1]

[16 25  8]

[ 1 71  7]

[12 74  0]

[ 7 62 10]

[13 46  3]

[28 32  4]

[ 6 88  2]

[25 23  0]

[18 55  3]

[20 65  5]

[30 77  1]

[5 3 2]

[18  0  4]

[20 77  3]

[11  6  0]

[28 52  6]

[10 85  0]

[28 70  7]

[29  2  2]

[14 76 10]

[18 21  8]

[ 4 75  3]

[23  7  0]

[23 77  8]

[ 9 72  4]

[17 75  2]

[23 76  9]

[30 43  0]

[ 0 20  3]

[22 30  8]

[13 36  1]

[ 9  7 10]

[ 9 45  4]

[ 7 68  3]

[29 57  3]

[22 82 10]

[25 13  6]

[ 1 10  7]

[ 0 23  3]

[28 81  5]

[17  7  3]

[ 8 24  2]

[24 74  4]

[13 20  4]

[19 32  0]

[15 12  3]

[10 65  3]

[25 60  8]

[ 8 24  3]

[30 82  5]

[7 2 6]

[ 3 10  7]

[ 6 54  5]

[21 82  1]

[17 86  7]

[25 70  0]

[ 3 66 10]

[ 4 71 10]

[24 48  2]

[17 54  8]

[11 15  2]

[12  5  1]

[26 17  4]

[20 42  0]

[16 20  4]

[13 48  1]

[19 22  7]

[ 9 13  3]

[18 53  1]

[15 84  6]

[ 0 10  6]

[ 4 55  9]

[25 61 10]

[15 56  6]

[ 2 89  9]

In [62]:
from pymoo.core.sampling import Sampling

class TopOrZeroSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        # vec = np.full((n_samples, 3), 10, dtype=np.int64)
        return [[30, 70, 0]]

algorithm = GA(
    sampling=TopOrZeroSampling(),
    eliminate_duplicates=True
)

# termination = get_termination('time', '00:00:05')

res = minimize(problem,
               algorithm,
               # termination,
               seed=1,
               verbose=False)

print(f'F: {res.F}, \nX: {res.X}')

[30 70  0]

[30.         71.81029402  0.        ]

[30.         73.80669468  0.        ]

[27.50164315 70.          0.        ]

[28.92108655 66.31113824  0.82134576]

[30.        70.         0.2581663]

[30.         67.89970382  0.51384148]

[30.         70.          0.24978205]

[28.09641925 70.          0.        ]

[30.         70.          1.34704585]

[30.         65.24260439  0.        ]

[30.         66.21053071  0.        ]

[29.69768558 64.89668317  0.        ]

[30.         70.          0.33268355]

[30.         66.37220355  0.        ]

[30.         71.36786967  0.        ]

[30.         70.91569937  0.        ]

[29.25790519 70.          0.29191813]

[30.         73.46451023  0.        ]

[29.66548311 67.83828938  0.        ]

[30.         64.47439116  0.        ]

[28.76262774 70.          0.        ]

[30.       70.068066  0.      ]

[30.         63.61358932  0.        ]

[30.         65.66142529  0.        ]

[30.         73.55777343  0.        ]

[30.         65.938796